In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

In [140]:
# reading files
commune=pd.read_excel(r"commune.xlsx")
enroll = pd.read_csv(r"enroll.csv")
quest = pd.read_csv(r"quest.csv")
industry = pd.read_csv(r"industry.csv")
ord = pd.read_csv(r"ord.csv")
study_domain = pd.read_csv(r"study_domain.csv")
transaction = pd.read_csv(r"transaction.csv")
technology = pd.read_csv(r"technology.csv")

In [197]:
commune.head()

,Commune_en,Commune_FR,commune,Departement,ADM1_PCODE
0,Abricots,Abricots,ht0812,Grande'Anse,HT08
1,Acul du Nord,Acul du Nord,ht0321,North,HT03
2,Anse-a-Foleur,Anse-à-Foleur,ht0922,North-West,HT09
3,Anse-a-Pitre,Anse-à-Pître,ht0234,South-East,HT02
4,Anse-a-Veau,Anse-à-Veau,ht1021,Nippes,HT10


Use all the files to have a final datasets to have the following columns
   <ul>
    <li>questid</li>
    <li>gender</li>
    <li>age (age of applicant)</li>
    <li>communeName (use data prep in commune file)</li>
    <li>application_date (created at in quest file)</li>
    <li>enroll_date (created_at in enroll file)</li>
    <li>is_enroll (Yes/No) (use data prep in enroll file)</li>
    <li>Education Level</li>
   <li>Communication channels(hear_AA1)</li>
    <li>Bootcamp Insterest (after_AA)</li>
    <li>Payement Date (use ord and transaction files)</li>
    <li>Payed (Yes/No)</li>
    <li>list Technologies as columns based (use get_dummies)</li>
    <li>list  Study domains (use get_dummies)</li>
    <li>Job is formal</li>
    <li>Have computer at home</li>
    <li>Have internet at home</li>
    
   </ul>
   

## Data Processing

In [156]:
# list  Study domains (use get_dummies)
study_domainok1 = pd.get_dummies(data=study_domain[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
study_domainok2 = study_domainok1.groupby("quest_id").sum()

In [157]:
#list Technologies as columns based (use get_dummies)
technologyok1 = pd.get_dummies(technology[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
technologyok2 = technologyok1.groupby("quest_id").sum()

In [143]:
# list industry (use get_dummies)
industryok1= pd.get_dummies(industry[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
industryok2= industryok1.groupby("quest_id").sum()

In [188]:
# Merge Tables by "quest_id"
resultok=pd.merge(left=quest, right=enroll[["user_id","percentage_completed",'quest_id']], how = 'left',on='quest_id')
resultok1=pd.merge(left=resultok, right=study_domainok2, how = 'left',on='quest_id')
resultok2=pd.merge(left=resultok1, right=technologyok2, how = 'left',on='quest_id')
resultok3=pd.merge(left=resultok2, right=industryok2, how = 'left',on='quest_id')


In [191]:
#Merge "transaction" by "user_id"
resultok5 = pd.merge(left=resultok4, right=transaction[['user_id','transaction_id']], how = 'left',on='user_id')
#Merge "Ord" by "quest_id"
resultok6 = pd.merge(left=resultok5, right=ord[['status',"amount_dollars",'quest_id']], how = 'left',on='quest_id')

In [198]:
# Merge Result3 and Commune
resultok7=pd.merge(left=resultok6, right=commune[["Commune_FR","Departement", "commune"]], how = 'left',on='commune')

In [203]:
resultok7.shape

(250, 59)

In [204]:
resultok7.columns

Index(['Unnamed: 0', 'gender', 'dob', 'commune', 'created_at', 'modified_at',
       'department', 'education_level', 'university', 'study_domain',
       'current_employed', 'formal_sector_job', 'have_computer_home',
       'internet_at_home', 'hear_AA_1', 'after_AA', 'quest_id', 'user_id',
       'percentage_completed', 'Accounting', 'Computer Science', 'Economics',
       'Electrical Engineering', 'Law', 'Management', 'Medicine', 'Statistics',
       'other_x', 'Bash_x', 'Excel_x', 'Git_x', 'Java_x', 'JavaScript_x',
       'PHP_x', 'PowerBI or Tableau_x', 'Python_x', 'R_x', 'SQL_x', 'VBA_x',
       'other_y', 'Bash_y', 'Excel_y', 'Git_y', 'Java_y', 'JavaScript_y',
       'PHP_y', 'PowerBI or Tableau_y', 'Python_y', 'R_y', 'SQL_y', 'VBA_y',
       'other', 'Commune_FR_x', 'Departement_x', 'transaction_id', 'status',
       'amount_dollars', 'Commune_FR_y', 'Departement_y'],
      dtype='object')

In [205]:
#drop unnecessary columns
resultok7.drop(["Unnamed: 0","modified_at","transaction_id","user_id","department","study_domain","user_id","transaction_id", "Bash_x", "Excel_x", "Git_x", "Java_x", "JavaScript_x",
       "PHP_x", "PowerBI or Tableau_x", "Python_x", "R_x", "SQL_x", "VBA_x",
       "other_y", "Bash_y", "Excel_y", "Git_y", "Java_y", "JavaScript_y",
       "PHP_y", "PowerBI or Tableau_y", "Python_y", "R_y", "SQL_y", "VBA_y",
       "other",], axis="columns" , inplace=True)
resultok7.head()

,gender,dob,commune,created_at,education_level,university,current_employed,formal_sector_job,have_computer_home,internet_at_home,...,Management,Medicine,Statistics,other_x,Commune_FR_x,Departement_x,status,amount_dollars,Commune_FR_y,Departement_y
0,male,25/08/1998,ht0111,2021-02-14T23:03:52.768400Z,unknown,unknown,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,Port-au-Prince,West,NaN,NaN,Port-au-Prince,West
1,female,29/09/1996,ht0111,2021-02-16T16:01:10.861844Z,Bachelors (bacc +4),other,No,unknown,Yes,Yes,...,0.0,0.0,0.0,1.0,Port-au-Prince,West,NaN,NaN,Port-au-Prince,West
2,male,17/05/1996,ht0111,2021-02-16T16:56:30.579504Z,Bachelors (bacc +4),Université d'Etat d'Haïti (UEH),No,unknown,Yes,Yes,...,NaN,NaN,NaN,NaN,Port-au-Prince,West,NaN,NaN,Port-au-Prince,West
3,male,24/11/1996,ht0111,2021-02-16T16:57:55.754541Z,Bachelors (bacc +4),Ecole Supérieure d'Infotronique d'Haïti (ESIH),No,unknown,Yes,Yes,...,1.0,0.0,0.0,0.0,Port-au-Prince,West,NaN,NaN,Port-au-Prince,West
4,male,20/07/1997,ht0111,2021-02-16T17:05:40.907948Z,High School (16 years),unknown,No,unknown,Yes,Yes,...,0.0,0.0,0.0,1.0,Port-au-Prince,West,NaN,NaN,Port-au-Prince,West


In [206]:
# Choose the 10 first Character before convert in Date
resultok7["created_at"] = resultok["created_at"].str[:10]

In [208]:
resultok7["dob"].replace({"3 aout 1977": "3/08/1977"}, inplace=True)
resultok7["dob"] = pd.to_datetime(resultok7["dob"])

In [209]:
#Convert "created_at" in format date

resultok7["created_at"] = pd.to_datetime(resultok7["created_at"])

In [210]:

# the Date 
#Create a Columns "age"
resultok7["age"] = ((resultok7["created_at"]-resultok7["dob"])//365)
#Choose the two first character in Age
resultok7["age"] = resultok7["age"].apply(lambda x: str(x)[:2])

In [211]:
resultok7.columns

Index(['gender', 'dob', 'commune', 'created_at', 'education_level',
       'university', 'current_employed', 'formal_sector_job',
       'have_computer_home', 'internet_at_home', 'hear_AA_1', 'after_AA',
       'quest_id', 'percentage_completed', 'Accounting', 'Computer Science',
       'Economics', 'Electrical Engineering', 'Law', 'Management', 'Medicine',
       'Statistics', 'other_x', 'Commune_FR_x', 'Departement_x', 'status',
       'amount_dollars', 'Commune_FR_y', 'Departement_y', 'age'],
      dtype='object')

In [212]:
#rename the columns
resultok7.rename(columns={'hear_AA_1':"Communication_Channel","after_AA":"Bootcamp_Interest","other_x":"Other_Study_Domain","created_at":"Inscription_Date"}, inplace=True)

In [213]:
resultok7.columns

Index(['gender', 'dob', 'commune', 'Inscription_Date', 'education_level',
       'university', 'current_employed', 'formal_sector_job',
       'have_computer_home', 'internet_at_home', 'Communication_Channel',
       'Bootcamp_Interest', 'quest_id', 'percentage_completed', 'Accounting',
       'Computer Science', 'Economics', 'Electrical Engineering', 'Law',
       'Management', 'Medicine', 'Statistics', 'Other_Study_Domain',
       'Commune_FR_x', 'Departement_x', 'status', 'amount_dollars',
       'Commune_FR_y', 'Departement_y', 'age'],
      dtype='object')

In [214]:
resultok7.shape

(250, 30)

###  1. How many observations and variables are there in the dataset
 

In [3]:
# here your codes

### 2.A Calculate the average age of the observations
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset
### 2.B Calculate 
   * variance of the whole datatset,male and female
   * kurtosis of the whole datatset,male and female
   * skewness of the whole datatset,male and female
### 2.C Does the age distribution follow a normal distribution 
   

In [4]:
# here your codes

### 3.Display the age frequency disbrution
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset

### 4. Can we say that the average age (24 years old) of the observations can be considered as the average age of the population likely to participate in this bootcamp. Justify your answer

In [5]:
#here your codes

### 5. Calculate the average age of participants for each communication channel

In [6]:
#here your codes

### 6. Display an age boxplot for each communication channel

In [7]:
#here your codes

### 7 .Is there a significant age difference between these groups

In [8]:
#here your codes

### 8.Plan a BA strategy for each communication channel regarding the age group

In [9]:
#here your codes

### 9.According to the observations what is the probability of being in each channel of communication knowing your are a woman

In [10]:
#here your codes

### 10.According to the observations what is the probability of being in each channel of communication knowing your are a man

In [11]:
#here your codes

### 11. Deduce the probability of being a woman knowing each communication channel

In [12]:
#here your codes

### 13. Deduce the probability of being a woman knowing each communication channel

In [13]:
# here your codes

### 14. Deduce the probability of being a man knowing each communication channel

In [14]:

#here your codes

### 15 Display a plot to see Gender vs Communication Channels .Is there any dependency between communication channels and gender?

In [15]:
#here 

### 16 Use the same method to display plot and know if is there any dependency between communication channels and Bootcamp Insterest?

In [16]:
# here your codes

### 17.Plan a BA  strategy for each communication channel, Bootcamp Insterest regarding the gender 

In [17]:
#here your codes

### 18.Plan a BA  strategy for each communication channel, Bootcamp Insterest regarding the gender

In [18]:
# here your codes

### 19. Calculate 
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=yes)
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=no)

### 20 reduce 
* P(Payed="yes"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")
* P(Payed="no"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")

In [19]:
# here your codes

### Based on these findings, propose strategies to increase our sales?

In [20]:
#here your codes